17102022

Бруцкий-Стемпковский

v. 02


Подготовить данные для этапа моделирования (обучения)
    
1. Анализ данных, необходимые корректировки. 
2. Обработать пропуски. 
3. Оценить выбросы. 
4. Корреляция. 
5. Тест на нормальность распределения.
6. Масштабировать и стандартизировать данные.
7. Разделите данные на тренрровачную и тестовую выборку.
8. При обработке пропусков/выбросов/скалировании тестовую и тренировачную выборку обрабатывать отдельно.

Необходимые импорты библиотек.

In [11]:
import numpy as np
import pandas as pd
import seaborn as sns
import pymorphy2

Прочитаем фрейм. Переименуем колонки.

In [3]:
columns = [ 
            'id', 'gender', 'age', 
            'marital_status', 'job', 
            'sum_of_credit', 'credit_term',
            'tarrif_id', 'score_shk', 'edu', 
            'region', 'income', 'credit_count', 
            'overdue_credit_count', 'flag'
            ]
data = pd.read_csv('credit_train_.csv', names=columns, skiprows=1, encoding='utf8', sep=';', thousands=",")
data.set_index("id", inplace=True)
#data.drop(['id'], axis = 1, inplace=True)
data.head()

,gender,age,marital_status,job,sum_of_credit,credit_term,tarrif_id,score_shk,edu,region,income,credit_count,overdue_credit_count,flag
id,,,,,,,,,,,,,,
1,M,NaN,NaN,UMN,5999800.0,10,1.6,NaN,GRD,КРАСНОДАРСКИЙ КРАЙ,30000.0,1.0,1.0,0
2,F,NaN,MAR,UMN,1088900.0,6,1.1,NaN,NaN,МОСКВА,NaN,2.0,0.0,0
3,M,32.0,MAR,SPC,1072800.0,12,1.1,NaN,NaN,ОБЛ САРАТОВСКАЯ,NaN,5.0,0.0,0
4,F,27.0,NaN,SPC,1200909.0,12,1.1,NaN,NaN,ОБЛ ВОЛГОГРАДСКАЯ,NaN,2.0,0.0,0
5,M,45.0,NaN,SPC,NaN,10,1.1,421385.0,SCH,ЧЕЛЯБИНСКАЯ ОБЛАСТЬ,NaN,1.0,0.0,0


Проверим информацию о строках.

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 170746 entries, 1 to 170746
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   gender                170746 non-null  object 
 1   age                   170743 non-null  float64
 2   marital_status        170743 non-null  object 
 3   job                   170746 non-null  object 
 4   sum_of_credit         170744 non-null  float64
 5   credit_term           170746 non-null  int64  
 6   tarrif_id             170746 non-null  float64
 7   score_shk             170739 non-null  float64
 8   edu                   170741 non-null  object 
 9   region                170554 non-null  object 
 10  income                170741 non-null  float64
 11  credit_count          161516 non-null  float64
 12  overdue_credit_count  161516 non-null  float64
 13  flag                  170746 non-null  int64  
dtypes: float64(7), int64(2), object(5)
memory usage: 19.

Фрейм содержит 14 признаков. Как числовые, так и категориальные признаки. Целевой показатель (функция от всех остальных аргументов) зашит в последней колонке 'flag'.

Большое количество выбросов сдержится в категориях о количестве имеющихся непогашенных и просроченных кредитов. От выбросов в остальных колонка просто избавимся.

Можно было бы заменить значения отдельных столбцов на среднее для числовых значений (применим это для столбиков с кредитами), либо заменить на самое часто встречающееся для категориальных значений.

In [5]:
drop_columns = [ 
            'gender', 'age', 'marital_status', 'job', 'sum_of_credit', 'credit_term',
            'tarrif_id', 'score_shk', 'edu', 'region', 'income', 'flag'
            ]
data.dropna(subset=drop_columns, inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 170543 entries, 8 to 170746
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   gender                170543 non-null  object 
 1   age                   170543 non-null  float64
 2   marital_status        170543 non-null  object 
 3   job                   170543 non-null  object 
 4   sum_of_credit         170543 non-null  float64
 5   credit_term           170543 non-null  int64  
 6   tarrif_id             170543 non-null  float64
 7   score_shk             170543 non-null  float64
 8   edu                   170543 non-null  object 
 9   region                170543 non-null  object 
 10  income                170543 non-null  float64
 11  credit_count          161331 non-null  float64
 12  overdue_credit_count  161331 non-null  float64
 13  flag                  170543 non-null  int64  
dtypes: float64(7), int64(2), object(5)
memory usage: 19.

In [6]:
data.head()

,gender,age,marital_status,job,sum_of_credit,credit_term,tarrif_id,score_shk,edu,region,income,credit_count,overdue_credit_count,flag
id,,,,,,,,,,,,,,
8,F,26.0,UNM,SPC,4787800.0,10,1.1,512525.0,GRD,МОСКОВСКАЯ ОБЛ,60000.0,3.0,0.0,0
10,F,32.0,UNM,SPC,2626800.0,10,1.1,465026.0,GRD,КРАЙ СТАВРОПОЛЬСКИЙ,39500.0,7.0,0.0,0
11,M,26.0,MAR,SPC,1679300.0,14,1.0,445430.0,SCH,САНКТ-ПЕТЕРБУРГ,36000.0,2.0,0.0,0
13,M,37.0,MAR,SPC,4228900.0,10,1.6,691609.0,SCH,ОБЛ АРХАНГЕЛЬСКАЯ,70000.0,1.0,0.0,0
14,M,43.0,MAR,INP,5756700.0,10,1.1,341164.0,GRD,ХАНТЫ-МАНСИЙСКИЙ АО,60000.0,7.0,0.0,0


С помощью метода .unique() исследуем отдельные колонки на нехарактерные значения.

In [7]:
file_obj = open("uniq_values.txt", "w")
columns = [ 'gender', 'age', 
            'marital_status', 'job', 
            'credit_term', 'tarrif_id', 'edu', 
            'region', 'credit_count', 
            'overdue_credit_count', 'flag'
            ]
for index in columns:
    file_obj.write(f"\n{index}\n\n")
    [file_obj.write(str(data[index].unique()[_]) + "\n") for _ in range(len(data[index].unique()))]
file_obj.close()

По результатам анализа видно следующее:

1. Колонки 'gender', 'age', 'marital_status', 'job', 'credit_term', 'tarrif_id', 'edu', 'income', 'flag'
очищены от испорченных данных. То есть их тип соотвествует ожидаемому, нет выбросов.

2. Колонки 'sum_of_credit' и 'score_shk' имеют большое число пропусков, но заменять пропуски мы будем после разбивки на тренировочоный и тестовый сеты.

3. Колонка 'region' имеет 301 уникальное значение, представляющее собой как строковые записи (повторяются по своей сути, но не по значению), встречаются числовые значения (74 и 98), вероятно представляющие собой номера регионов (Челябинск и Санкт-Петербург соответсвенно).

С последними поработаем отдельно. Выпишем их в отельный файл для визуального анализа.

In [8]:
file_obj = open("region.txt", "w")

file_obj.write(f"region\n\n")
[file_obj.write(str(data['region'].unique()[_]) + "\n") for _ in range(len(data['region'].unique()))]
file_obj.close()

In [ ]:
morph = pymorphy2.MorphAnalyzer()
text_list_lemm = []

for i in range(len(data['region'].unique())):
    text_list_lemm.append(morph.parse(data['region'].unique()[i])[0].normal_form)

text_list_lemm